In [5]:
import os
import numpy as np
import pandas as pd
from shutil import copyfile
# os.mkdir('./matches/')
# alpha = 'a'
# for i in range(0): 
#     os.mkdir('./images/' + alpha)
#     alpha = chr(ord(alpha) + 1)

rootdir = './data/'
# k=1

# for file in os.listdir(rootdir):
#     letter = file[0]
#     os.mkdir('./matches/' + str(k))
#     copyfile(rootdir+file, './matches/' + str(k)+'/'+str(k))
#     k+=1


In [6]:
l=1
scores=np.zeros(50,dtype=int)
groundsdata = []
teamwisescores=np.zeros(20,dtype=int)
teams=[]
def firstinningscore(balls,scorebat,scoreextras):
    fscore = 0
    length = balls.shape[0]
    for i in range(200):
        if(balls[i]<balls[i+1]):
            fscore += (scorebat[i]+ scoreextras[i])
        else:
            fscore += (scorebat[i]+ scoreextras[i])
            return fscore

for file in os.listdir(rootdir):
    df= pd.read_csv('./matches/'+str(l)+'/'+str(l))
    # print(l)
    if(l==507 or l==752):
        l+=2
    else:
        l+=1
    if(l==842):
        break
    venues = df['venue']
    balls = df['ball']
    battingteam = df['batting_team']
    try:
        teams.index(battingteam[0])
    except:
        teams.append(battingteam[0])        
    scorebat = df['runs_off_bat']
    scoreextras = df['extras']
    v1 = venues.drop_duplicates(keep='first',inplace=False)    
    score = firstinningscore(balls,scorebat,scoreextras)
    teamindex = teams.index(battingteam[0])
    tscore=teamwisescores[teamindex]
    teamwisescores[teamindex]= int((tscore+score)/2)

    try :
        groundsdata.index(v1[0])
        ind = groundsdata.index(v1[0])
        gscore = scores[ind]
        scores[ind]=int((gscore+score)/2)
    except:
        groundsdata.append(v1[0])
        ind = groundsdata.index(v1[0])
        gscore = scores[ind]
        scores[ind]=int((gscore+score)/2)


In [8]:
totaldataset={
    "venue":[],
    "teamplaying":[],
    "groundavgscore":[],
    "teamavgscore":[],
    "matchscore":[]
}
l=1

for file in os.listdir(rootdir):
    df= pd.read_csv('./matches/'+str(l)+'/'+str(l))
    # print(l)
    if(l==507 or l==752):
        l+=2
    else:
        l+=1
    if(l==842):
        break
    venues = df['venue']
    v1 = venues.drop_duplicates(keep='first',inplace=False)  
    presentvenue = v1[0]
    balls = df['ball']
    scorebat = df['runs_off_bat']
    scoreextras = df['extras']
    score = firstinningscore(balls,scorebat,scoreextras)
    battingteam = df['batting_team']
    playingteam=battingteam[0]
    groundindex= groundsdata.index(v1[0])
    avgscorevenue= scores[groundindex]
    teamindex = teams.index(playingteam)
    teamavgscore = teamwisescores[teamindex]
    totaldataset["venue"].append(str(presentvenue))
    totaldataset["teamplaying"].append(str(playingteam))
    totaldataset["groundavgscore"].append(int(avgscorevenue))
    totaldataset["teamavgscore"].append(int(teamavgscore))
    totaldataset["matchscore"].append(int(score))
    


In [9]:
import pandas as pd 
from pandas import ExcelWriter
from pandas import ExcelFile
dataframe = pd.DataFrame(totaldataset)


In [10]:
writer = ExcelWriter('personaldataset.xlsx')
dataframe.to_excel(writer,'Sheet1',index=False)
writer.save()
dataframe.to_csv('personaldata.csv', index=False)

In [12]:
from sklearn import linear_model

df2 = pd.read_csv("./personaldata.csv")

X = df2[['groundavgscore', 'teamavgscore']].values.reshape(-1,2)
y = df2['matchscore']

regr = linear_model.LinearRegression()
model = regr.fit(X, y)



In [54]:
venueinput = str(input("please give the venue"))
teaminput = str(input("please give the team playing "))
vindex = groundsdata.index(venueinput)
vscore = scores[vindex]
tindex = teams.index(teaminput)
teamscore = teamwisescores[tindex]
predictscore = int(model.predict([[vscore,teamscore]]))
predictionlogistic = classifier.predict([[vscore,teamscore]])

print("The predicted score of first innings using linear is :",predictscore)
print("with runrate of :",(predictscore/20))
print("The predicted score of first innings using linear is :",predictionlogistic)
print("with runrate of :",(predictionlogistic/20))





The predicted score of first innings using linear is : 158
with runrate of : 7.9
The predicted score of first innings using linear is : [263]
with runrate of : [13.15]


In [90]:
regr.coef_

array([0.21789534, 0.00560069])

In [14]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)
predictionlogistic = classifier.predict([[vscore,teamscore]])


C:\Users\prabh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [44]:
length = dataframe[dataframe.columns[0]].count()

In [36]:
linear=[]
logistic =[]
for i in range(length):
    groundavgscore = X[i][0]
    teamscores = X[i][1]
    predlin=int(model.predict([[groundavgscore,teamscores]])) 
    plog= int(classifier.predict([[groundavgscore,teamscores]]))
    linear.append(predlin)
    logistic.append(plog)



In [49]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Gujarat Lions',
 'Rising Pune Supergiant',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Delhi Daredevils',
 'Kings XI Punjab',
 'Rajasthan Royals',
 'Chennai Super Kings',
 'Delhi Capitals',
 'Punjab Kings',
 'Deccan Chargers',
 'Kochi Tuskers Kerala',
 'Pune Warriors',
 'Rising Pune Supergiants']

In [50]:
groundsdata

['Rajiv Gandhi International Stadium, Uppal',
 'Maharashtra Cricket Association Stadium',
 'Saurashtra Cricket Association Stadium',
 'Holkar Cricket Stadium',
 'M.Chinnaswamy Stadium',
 'Wankhede Stadium',
 'Eden Gardens',
 'M Chinnaswamy Stadium',
 'Feroz Shah Kotla',
 'Punjab Cricket Association IS Bindra Stadium, Mohali',
 'Green Park',
 'Punjab Cricket Association IS Bindra Stadium',
 'Rajiv Gandhi International Stadium',
 'MA Chidambaram Stadium',
 'Sawai Mansingh Stadium',
 'Arun Jaitley Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Sheikh Zayed Stadium',
 'Dubai International Cricket Stadium',
 'Sharjah Cricket Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai',
 'Wankhede Stadium, Mumbai',
 'Narendra Modi Stadium, Ahmedabad',
 'Arun Jaitley Stadium, Delhi',
 'Punjab Cricket Association Stadium, Mohali',
 'MA Chidambaram Stadium, Chepauk',
 'Dr DY Patil Sports Academy',
 'Newlands',
 "St George's Park",
 'Kingsmead',
 'SuperSport Park',
 'Buffalo Park',